In [16]:
from models.reinforcement_learning import *

In [17]:
track_file = 'datasets/L-track.txt'
racetrack = Racetrack(track_file)

In [18]:
# Access the loaded track data, starts, and finishes directly from the instance
track = racetrack.track
starts = racetrack.starting_points
finishes  = racetrack.finish_lines

In [19]:
starts

[(1, 6), (1, 7), (1, 8), (1, 9)]

In [20]:
print(track[1][32])

F


In [21]:
# Create a function to test different aspects of the Simulator
def test_simulator():
    track_file = 'datasets/L-track.txt'
    racetrack = Racetrack(track_file)
    simulator = Simulator(racetrack, crash_option='nearest')

    print("Initial State:", (simulator.car.x, simulator.car.y, simulator.car.vx, simulator.car.vy))
    state, reward, done = simulator.step((0, 0))
    print("Test 1 - No Movement:", state, "Reward:", reward, "Done:", done)

    state, reward, done = simulator.step((1, 1))
    print("Test 2 - Max Acceleration:", state, "Reward:", reward, "Done:", done)

    while not done:
        state, reward, done = simulator.step((1, 0))
        if simulator.check_path_for_crash(simulator.car.x, simulator.car.y, simulator.car.x + 1, simulator.car.y):
            print("Crash Detected at:", state)
            break

    simulator.car.x, simulator.car.y = 0, 0
    state, reward, done = simulator.step((-1, -1))
    print("Boundary Test:", state, "Reward:", reward, "Done:", done)

    simulator.reset()
    print("State after reset:", (simulator.car.x, simulator.car.y, simulator.car.vx, simulator.car.vy))

test_simulator()


Initial State: (1, 9, 0, 0)
Test 1 - No Movement: (1, 9, 0, 0) Reward: -1 Done: False
Test 2 - Max Acceleration: (2, 9, 0, 0) Reward: -1 Done: False
Crash Detected at: (35, 9, 0, 0)
Boundary Test: (1, 6, 0, 0) Reward: -1 Done: False
State after reset: (1, 9, 0, 0)


In [22]:
def test_move_right_until_crash(track_file, crash_option):
    """Tests the simulator's behavior when a car moves right until it crashes.
    Verifies crash handling and simulator reset based on the crash_option.
    Limits the test to a maximum number of steps to prevent infinite loops.
    """

    racetrack = Racetrack(track_file)
    simulator = Simulator(racetrack, crash_option=crash_option)

    print(f"Testing with crash option: {crash_option}")
    print("Initial State:", (simulator.car.x, simulator.car.y, simulator.car.vx, simulator.car.vy))

    max_steps = 50  # Limit the test to a reasonable number of steps 

    for steps in range(max_steps):
        state, reward, done = simulator.step((1, 0))  # Move right
        print(f"Step {steps}: Position ({simulator.car.x}, {simulator.car.y}), Velocity ({simulator.car.vx}, {simulator.car.vy}), Reward {reward}")

        # Check for crash conditions
        if simulator.car.x >= len(racetrack.track[0]) - 1 or racetrack.track[simulator.car.y][simulator.car.x] == '#':
            print("Crash Detected. Handling crash...")
            break

        if done:  # Check for other termination conditions
            print("Simulation ended.")
            break

    # After crash, let's see where the car ends up
    print("Final State after crash:", (simulator.car.x, simulator.car.y, simulator.car.vx, simulator.car.vy))

    # Assertions based on crash_option 
    if crash_option == 'nearest':
        # Check if car moved to a valid track position and velocity is zero
        assert racetrack.track[simulator.car.y][simulator.car.x] != '#'
        assert simulator.car.vx == 0 and simulator.car.vy == 0
    elif crash_option == 'original':
        # Check if car is back at a starting position and velocity is zero
        assert (simulator.car.x, simulator.car.y) in racetrack.starting_points
        assert simulator.car.vx == 0 and simulator.car.vy == 0

    print("Resetting the simulator...")
    simulator.reset() 
    print("State after reset:", (simulator.car.x, simulator.car.y, simulator.car.vx, simulator.car.vy))

    # Add an assertion to check reset position
    assert (simulator.car.x, simulator.car.y) in racetrack.starting_points 
    assert simulator.car.vx == 0 and simulator.car.vy == 0

    print("-" * 50)

# Test the function with both crash options
track_file = 'datasets/L-track.txt'
test_move_right_until_crash(track_file, crash_option='nearest')
test_move_right_until_crash(track_file, crash_option='original') 


Testing with crash option: nearest
Initial State: (1, 7, 0, 0)
Step 0: Position (2, 7), Velocity (1, 0), Reward -1
Step 1: Position (3, 7), Velocity (1, 0), Reward -1
Step 2: Position (5, 7), Velocity (2, 0), Reward -1
Step 3: Position (8, 7), Velocity (3, 0), Reward -1
Step 4: Position (11, 7), Velocity (3, 0), Reward -1
Step 5: Position (15, 7), Velocity (4, 0), Reward -1
Step 6: Position (19, 7), Velocity (4, 0), Reward -1
Step 7: Position (24, 7), Velocity (5, 0), Reward -1
Step 8: Position (29, 7), Velocity (5, 0), Reward -1
Step 9: Position (34, 7), Velocity (5, 0), Reward -1
Step 10: Position (35, 7), Velocity (0, 0), Reward -1
Step 11: Position (35, 7), Velocity (0, 0), Reward -1
Step 12: Position (35, 7), Velocity (0, 0), Reward -1
Step 13: Position (35, 7), Velocity (0, 0), Reward -1
Step 14: Position (35, 7), Velocity (0, 0), Reward -1
Step 15: Position (35, 7), Velocity (0, 0), Reward -1
Step 16: Position (35, 7), Velocity (0, 0), Reward -1
Step 17: Position (35, 7), Veloci

AssertionError: 